In [8]:
import dask.bag as db
import dask.dataframe as dd  # you can use Dask bags or dataframes
from os import path
import nltk, csv
from nltk.stem import PorterStemmer 

filename_train = "../dataset/train.csv"
filename_test = "../dataset/valid.csv"

In [42]:
dd.read_csv(filename_test, escapechar='"', quotechar='"', quoting=csv.QUOTE_NONE, encoding='utf-8').compute()

ParserError: Error tokenizing data. C error: Expected 6 fields in line 296, saw 7
